In [22]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, SGDClassifier, Lasso,RidgeCV
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, learning_curve, KFold
from sklearn.dummy import DummyRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, median_absolute_error, mean_absolute_percentage_error, RocCurveDisplay, mean_squared_log_error,  accuracy_score,precision_score,recall_score,f1_score
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, RobustScaler, StandardScaler, FunctionTransformer, LabelEncoder, OneHotEncoder, Binarizer, OrdinalEncoder, MaxAbsScaler
from sklearn.compose import make_column_transformer
from sklearn.decomposition import PCA 
from sklearn.impute import SimpleImputer  
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectFromModel, RFE, RFECV 
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor, ExtraTreesRegressor, VotingClassifier, VotingRegressor,StackingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
# from pycaret.regression import *
import xgboost
from os import stat
import numpy as np
import pandas as pd 
from contextlib import closing
from urllib.request import urlopen
import json
from datetime import datetime 
import lightgbm


import warnings
warnings.filterwarnings('ignore')


In [23]:
def transform_weather(x):
    if x == 1:
        x = 4
    elif x == 2:
        x = 3
    elif x == 3:
        x = 2 
    elif x == 4:
        x = 1
        
    return x 

In [24]:
def transform_season(x):
    if x == 1:
        x = 3
    elif x == 2:
        x = 4
    elif x == 3:
        x = 2 
    elif x == 4:
        x = 1
        
    return x 

# Les transformations : 

Transformation des variables weather & season
(1 à 4) => 1 les mauvais jours et 4 les meilleurs


Ajout de data avec 0 count aux températures extrême pour donner une info type catastrophe naturelle. 

Pour insister sur le fait que le pic de fréquentation est aux alentours des 30 degrès et n'augmentera pas s'il fait "trop" chaud


Transformation colonne datetime en dayofyear, hour et year. 


In [25]:
# Les transformations 
train_data_all =  pd.read_csv('../data/train.csv')
train_data_all.drop('Unnamed: 0', axis=1, inplace=True)

train_data_all = train_data_all.append({'datetime' : '2011-08-17 08:00:00', 'season' : 0, 'holiday' : 0, 'workingday' : 0, 'weather' : 4, 'temp' : 75, 'atemp' : 72, 'humidity' : 2, 'windspeed' : 500, 'casual' : 0, 'registered' : 0, 'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2012-07-15 09:00:00', 'season' : 0, 'holiday' : 0, 'workingday' : 0, 'weather' : 1, 'temp' : 96, 'atemp' : 88, 'humidity' : 2, 'windspeed' : 500.0, 'casual' : 0, 'registered' : 0, 'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2011-08-10 11:00:00', 'season' : 1, 'holiday' : 0, 'workingday' : 0, 'weather' : 4, 'temp' : 200, 'atemp' : 155, 'humidity' : 51, 'windspeed' : 0, 'casual' : 0, 'registered' : 0, 'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2011-08-12 19:00:00', 'season' : 2, 'holiday' : 0, 'workingday' : 0, 'weather' : 3, 'temp' : 111, 'atemp' : 110, 'humidity' : 4, 'windspeed' : 225, 'casual' : 0, 'registered' : 0, 'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2011-12-08 18:00:00', 'season' : 4, 'holiday' : 0, 'workingday' : 0, 'weather' : 4, 'temp' : -48, 'atemp' : -55, 'humidity' : 9, 'windspeed' : 110, 'casual' : 0, 'registered' : 0,  'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2011-12-24 17:00:00', 'season' : 4, 'holiday' : 0, 'workingday' : 0, 'weather' : 4, 'temp' : -110, 'atemp' : -108, 'humidity' : 25, 'windspeed' : 110, 'casual' : 0, 'registered' : 0,  'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2011-11-25 16:00:00', 'season' : 4, 'holiday' : 0, 'workingday' : 0, 'weather' : 4, 'temp' : -48, 'atemp' : -55, 'humidity' : 69, 'windspeed' : 110, 'casual' : 0, 'registered' : 0,  'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2011-12-27 15:00:00', 'season' : 4, 'holiday' : 0, 'workingday' : 0, 'weather' : 1, 'temp' : -500, 'atemp' : -55, 'humidity' : 0, 'windspeed' : 110, 'casual' : 0, 'registered' : 0,  'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2011-07-21 15:00:00', 'season' : 0, 'holiday' : 0, 'workingday' : 0, 'weather' : 2, 'temp' : 75, 'atemp' : 72.5, 'humidity' : 41, 'windspeed' : 500, 'casual' : 0, 'registered' : 0, 'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2011-06-04 13:00:00', 'season' : 1, 'holiday' : 0, 'workingday' : 0, 'weather' : 4, 'temp' : 200, 'atemp' : 155, 'humidity' : 55, 'windspeed' : 0, 'casual' : 0, 'registered' : 0, 'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2011-06-05 14:00:00', 'season' : 2, 'holiday' : 0, 'workingday' : 0, 'weather' : 3, 'temp' : 111, 'atemp' : 110, 'humidity' : 25, 'windspeed' : 225, 'casual' : 0, 'registered' : 0, 'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2011-11-28 15:00:00', 'season' : 4, 'holiday' : 0, 'workingday' : 0, 'weather' : 4, 'temp' : -48, 'atemp' : -55, 'humidity' : 39, 'windspeed' : 110, 'casual' : 0, 'registered' : 0, 'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2011-12-27 09:00:00', 'season' : 4, 'holiday' : 0, 'workingday' : 0, 'weather' : 3, 'temp' : -110, 'atemp' : -108, 'humidity' : 10, 'windspeed' : 110, 'casual' : 0, 'registered' : 0, 'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2011-11-27 12:00:00', 'season' : 4, 'holiday' : 0, 'workingday' : 0, 'weather' : 4, 'temp' : -48, 'atemp' : -55, 'humidity' : 57, 'windspeed' : 110, 'casual' : 0, 'registered' : 0, 'count' : 0}, ignore_index = True)
train_data_all = train_data_all.append({'datetime' : '2012-01-02 16:00:00', 'season' : 4, 'holiday' : 0, 'workingday' : 0, 'weather' : 4, 'temp' : -500, 'atemp' : -55, 'humidity' : 84, 'windspeed' : 110, 'casual' : 0, 'registered' : 0, 'count' : 0}, ignore_index = True)

train_data_all['day'] = pd.to_datetime(train_data_all['datetime']).dt.dayofyear
train_data_all['hour'] = pd.to_datetime(train_data_all['datetime']).dt.hour
train_data_all['year'] = pd.to_datetime(train_data_all['datetime']).dt.year

train_data_all.drop(['datetime'], axis=1, inplace=True)


for i in range(0, len(train_data_all.iloc[:,0])) :
    train_data_all.iloc[i,0] = transform_season(int(train_data_all.iloc[i,0] ))
    
for i in range(0, len(train_data_all.iloc[:,3])) :
    train_data_all.iloc[i,3] = transform_weather(int(train_data_all.iloc[i,3] ))


train_data_all = train_data_all.drop(['registered','casual'], axis=1)



augmentation = np.linspace(0, (144 * 100 / 230) *  train_data_all.shape[0] , train_data_all.shape[0])

# train_data_all['augmentation'] = 0 
# train_data_all['augmentation'] = augmentation # Etape suivante : Il faut order par day avant de drop datetime pour que ce soit cohérent 




train_data_all_X = train_data_all.drop('count', axis = 1)
train_data_all_y = train_data_all['count']




test_data_all =  pd.read_csv('../data/test.csv')
test_data_all.drop('Unnamed: 0', axis=1, inplace=True)
test_data_all['day'] = pd.to_datetime(test_data_all['datetime']).dt.dayofyear
test_data_all['hour'] = pd.to_datetime(test_data_all['datetime']).dt.hour
test_data_all['year'] = pd.to_datetime(test_data_all['datetime']).dt.year
test_data_all.drop(['datetime'], axis=1, inplace=True)

for i in range(0, len(test_data_all.iloc[:,0])) :
    test_data_all.iloc[i,0] = transform_season(int(test_data_all.iloc[i,0] ))
    
for i in range(0, len(test_data_all.iloc[:,3])) :
    test_data_all.iloc[i,3] = transform_weather(int(test_data_all.iloc[i,3] ))

# augmentation = np.linspace(0, (144 * 100 / 230) *  test_data_all.shape[0] / 100 , test_data_all.shape[0])
# test_data_all['augmentation'] = augmentation


test_data_all = test_data_all.drop(['registered','casual'], axis=1)

test_data_all_X = test_data_all.drop('count', axis = 1)
test_data_all_y = test_data_all['count']


On utilise deux pipelines, l'une pour traité les valeurs numériques et l'autres pour les valeurs catégoriques. 

In [ ]:
# Meilleur model : 
# Preprocessing et model xgboost :0.9056841627368329(8s)

logfunc = FunctionTransformer(func=np.log1p, inverse_func=np.exp, check_inverse=False)

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline( SimpleImputer(), MaxAbsScaler())
encode_pipeline = make_pipeline(StandardScaler(with_mean=False, with_std=True), OneHotEncoder(handle_unknown = 'ignore'))

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))

xgb = xgboost.XGBRegressor(booster='gbtree', colsample_bytree = 0.493659, eta = 0.28, gamma=0,max_depth = 7, max_leaves=0, n_estimators =100, objective='reg:squarederror', random_state=32, 
                            reg_alpha=1.6666666666666667, reg_lambda =1.9791666666666667, subsample_freq=0.75, tree_method='auto')

model_xgboost = make_pipeline(preprocessor,  xgb)
model_xgboost.fit(train_data_all_X , train_data_all_y)
print('train data', model_xgboost.score(train_data_all_X, train_data_all_y))
print('test data', model_xgboost.score(test_data_all_X, test_data_all_y))

[10:01:25] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


train data 0.986720359083927
test data 0.8967965326020695


In [67]:
# Preprocessing et model lgbm : 0.9190960274167893 (8s)

logfunc = FunctionTransformer(func=np.log1p, inverse_func=np.exp, check_inverse=False)

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline( logfunc, SimpleImputer(missing_values=np.nan), MaxAbsScaler())
encode_pipeline = make_pipeline(StandardScaler(with_mean=False, with_std=True), OneHotEncoder(handle_unknown = 'ignore'))

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))


model_lgbm =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.7395, learning_rate=0.12, max_bin=1023, max_depth=8, reg_lambda=1,reg_alpha=0, subsample_freq=7)
model_lgbm_fit = make_pipeline(preprocessor, model_lgbm)
model_lgbm_fit.fit(train_data_all_X , train_data_all_y)
print('train', model_lgbm_fit.score(train_data_all_X, train_data_all_y))
print('test', model_lgbm_fit.score(test_data_all_X, test_data_all_y))

train 0.9614442590972704
test 0.9190960274167893


In [69]:
# Preprocessing et model lgbm : 0.9190960274167893 (8s)

logfunc = FunctionTransformer(func=np.log1p, inverse_func=np.exp, check_inverse=False)

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline(  SimpleImputer(), MaxAbsScaler())
encode_pipeline = make_pipeline(StandardScaler(with_mean=False, with_std=True), OneHotEncoder(handle_unknown = 'ignore'))

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))


model_lgbm_2 =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.7385, learning_rate=0.07, max_bin=725, max_depth=8, reg_lambda=0.68596,reg_alpha=0, subsample_freq=56)
model_lgbm_fit = make_pipeline(preprocessor, model_lgbm_2)
model_lgbm_fit.fit(train_data_all_X , train_data_all_y)
print('train', model_lgbm_fit.score(train_data_all_X, train_data_all_y))
print('test', model_lgbm_fit.score(test_data_all_X, test_data_all_y))

train 0.9521333672398784
test 0.9051328597939277


In [19]:
# Meilleur model n2 lgbm : 
# Preprocessing et model lgbm :0.9118832634446215 (8s)

logfunc = FunctionTransformer(func=np.log1p, inverse_func=np.exp, check_inverse=False)

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline( logfunc, SimpleImputer(), MaxAbsScaler())
encode_pipeline = make_pipeline(StandardScaler(with_mean=False, with_std=True), OneHotEncoder(handle_unknown = 'ignore'))

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))


model_lgbm_3 = lightgbm.LGBMRegressor(random_state=2, min_data_in_leaf=100, max_bin=1023, reg_alpha=0, subsample_freq=56)
model_lgbm_fit = make_pipeline(preprocessor,  model_lgbm_3)
model_lgbm_fit.fit(train_data_all_X , train_data_all_y)
print('train', model_lgbm_fit.score(train_data_all_X, train_data_all_y))
print('test', model_lgbm_fit.score(test_data_all_X, test_data_all_y))

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
train 0.9522756520038445
test 0.9118832634446215


In [20]:
# colsample_bytree => Pourcentage de features utilisé pour chaque arbre réduit a 46% et resultat pas loin de 73% + modif learning rate 
# Preprocessing et model lgbm : 0.9064545094483808(8s)

logfunc = FunctionTransformer(func=np.log1p, inverse_func=np.exp, check_inverse=False)

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline( logfunc, SimpleImputer(), MaxAbsScaler())
encode_pipeline = make_pipeline(StandardScaler(with_mean=False, with_std=True), OneHotEncoder())

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))


model_lgbm_4 =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.469, learning_rate=0.1239599, max_bin=1023, max_depth=8, reg_lambda=1,reg_alpha=0, subsample_freq=7)
model_lgbm_fit = make_pipeline(preprocessor, model_lgbm_4)
model_lgbm_fit.fit(train_data_all_X , train_data_all_y)
print('train', model_lgbm_fit.score(train_data_all_X, train_data_all_y))
print('test', model_lgbm_fit.score(test_data_all_X, test_data_all_y))

train 0.9588957920387866
test 0.9074659738499581


Modele => Voting Regressor 

In [21]:
# Preprocessing et model Votingmodel : 0.917337454042215 (2s4)

logfunc = FunctionTransformer(func=np.log1p, inverse_func=np.exp, check_inverse=False)

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline( logfunc, SimpleImputer(), MaxAbsScaler())
encode_pipeline = make_pipeline(StandardScaler(), OneHotEncoder())

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))

VC = VotingRegressor([('xgboos' , xgb),('lgbm' , model_lgbm), ('lgbm2' , model_lgbm_2),('lgbm3' , model_lgbm_3), ('lgbm4' , model_lgbm_4)])

model_voting = make_pipeline(preprocessor, VC)

model_voting.fit(train_data_all_X, train_data_all_y)
model_voting.score(test_data_all_X, test_data_all_y)

[22:11:18] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




0.9140491767414458

Model de stacking => 

In [23]:
# Preprocessing et model Votingmodel : 0.9288990012377096 (6s)

logfunc = FunctionTransformer(func=np.log1p, inverse_func=np.exp, check_inverse=False)

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline( logfunc, SimpleImputer(), MaxAbsScaler())
encode_pipeline = make_pipeline(StandardScaler(with_mean=False, with_std=True), OneHotEncoder(handle_unknown = 'ignore'))

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))

stack_model = StackingRegressor([('xgboos' , xgb),('lgbm' , model_lgbm), ('lgbm2' , model_lgbm_3)])

model_stack = make_pipeline(preprocessor, stack_model)

model_stack.fit(train_data_all_X, train_data_all_y)
model_stack.score(test_data_all_X, test_data_all_y)
y_pred = model_stack.predict(test_data_all_X)
print('mean abs error' , mean_absolute_error(test_data_all_y, y_pred))

[22:12:48] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:12:49] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:12:50] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually bei

In [12]:
# Preprocessing et model Votingmodel : 0.9288990012377096 (6s)

logfunc = FunctionTransformer(func=np.log1p, inverse_func=np.exp, check_inverse=False)

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline( logfunc, SimpleImputer(), MaxAbsScaler())
encode_pipeline = make_pipeline(StandardScaler(with_mean=False, with_std=True), OneHotEncoder(handle_unknown = 'ignore'))

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))

stack_model = StackingRegressor([('xgboos' , xgb),('lgbm' , model_lgbm), ('lgbm2' , model_lgbm_2),('lgbm3' , model_lgbm_3),('lgbm4' , model_lgbm_4)])

model_stack = make_pipeline(preprocessor, stack_model)

model_stack.fit(train_data_all_X, train_data_all_y)
model_stack.score(test_data_all_X, test_data_all_y)

[19:59:42] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:59:44] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:59:45] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually bei

0.9244984606061998

In [13]:
# Preprocessing et model Votingmodel : 0.9288990012377096 (6s)

logfunc = FunctionTransformer(func=np.log1p, inverse_func=np.exp, check_inverse=False)

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline( logfunc, SimpleImputer(), MaxAbsScaler())
encode_pipeline = make_pipeline(StandardScaler(with_mean=False, with_std=True), OneHotEncoder(handle_unknown = 'ignore'))

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))

stack_model = StackingRegressor([('xgboos' , xgb),('lgbm' , model_lgbm), ('lgbm2' , model_lgbm_2),('lgbm3' , model_lgbm_3),('lgbm4' , model_lgbm_4)])

model_stack = make_pipeline(preprocessor, stack_model)

model_stack.fit(train_data_all_X, train_data_all_y)
model_stack.score(test_data_all_X, test_data_all_y)

[19:59:54] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:59:56] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:59:57] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually bei

0.9244984606061998

# Les modèles avec une fonction logarythme donner de bons score mais elle pose un problème de prédictions avec les valeurs négatives. 

# On a donc décidé de retiré la fonction logarythme et d'optimiser les paramètres pour rehausser le score 

In [19]:
# Meilleur model : 
# Preprocessing et model xgboost :0.9056841627368329(8s)

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline( SimpleImputer(), MaxAbsScaler())
encode_pipeline = make_pipeline(StandardScaler(with_mean=False, with_std=True), OneHotEncoder(handle_unknown = 'ignore'))

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))

xgb = xgboost.XGBRegressor(booster='gbtree', colsample_bytree = 0.493659, eta = 0.28, gamma=0,max_depth = 7, max_leaves=0, n_estimators =100, objective='reg:squarederror', random_state=32, 
                            reg_alpha=1.6666666666666667, reg_lambda =1.9791666666666667, subsample_freq=0.75, tree_method='auto')

model_xgboost = make_pipeline(preprocessor,  xgb)
model_xgboost.fit(train_data_all_X , train_data_all_y)
print('train data', model_xgboost.score(train_data_all_X, train_data_all_y))
print('test data', model_xgboost.score(test_data_all_X, test_data_all_y))

[10:01:25] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


train data 0.986720359083927
test data 0.8967965326020695


In [13]:
# Preprocessing et model lgbm : 0.9190960274167893 (8s)

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline(  SimpleImputer(), MaxAbsScaler())
encode_pipeline = make_pipeline(StandardScaler(with_mean=False, with_std=True), OneHotEncoder(handle_unknown = 'ignore'))

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))

model_lgbm_2 =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.7385, learning_rate=0.07, max_bin=725, max_depth=8, reg_lambda=0.68596,reg_alpha=0, subsample_freq=56)
model_lgbm_fit = make_pipeline(preprocessor, model_lgbm_2)
model_lgbm_fit.fit(train_data_all_X , train_data_all_y)
print('train', model_lgbm_fit.score(train_data_all_X, train_data_all_y))
print('test', model_lgbm_fit.score(test_data_all_X, test_data_all_y))

train 0.9521333672398784
test 0.9051328597939277


In [14]:
# All models 

xgb = xgboost.XGBRegressor(booster='gbtree', colsample_bytree = 0.493659, eta = 0.28, gamma=0,max_depth = 7, max_leaves=0, n_estimators =100, objective='reg:squarederror', random_state=32, 
                            reg_alpha=1.6666666666666667, reg_lambda =1.9791666666666667, subsample_freq=0.75, tree_method='auto')

model_lgbm =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.7395, learning_rate=0.12, max_bin=1023, max_depth=8, reg_lambda=1,reg_alpha=0, subsample_freq=7)

model_lgbm_2 =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.7385, learning_rate=0.07, max_bin=725, max_depth=8, reg_lambda=0.68596,reg_alpha=0, subsample_freq=56)

model_lgbm_3 = lightgbm.LGBMRegressor(random_state=2, min_data_in_leaf=100, max_bin=1023, reg_alpha=0, subsample_freq=56)

model_lgbm_4 =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.469, learning_rate=0.1239599, max_bin=1023, max_depth=8, reg_lambda=1,reg_alpha=0, subsample_freq=7)

VC = VotingRegressor([('xgboos' , xgb),('lgbm' , model_lgbm), ('lgbm2' , model_lgbm_2),('lgbm3' , model_lgbm_3), ('lgbm4' , model_lgbm_4)])

stack_model = StackingRegressor([('xgboos' , xgb),('lgbm' , model_lgbm), ('lgbm2' , model_lgbm_2),('lgbm3' , model_lgbm_3),('lgbm4' , model_lgbm_4)])



# Test d'optimisation avec Optuna 

In [38]:
import optuna 
def objective(trial):
    import xgboost as xgb 
    dtrain = xgb.DMatrix(train_data_all_X, label=train_data_all_y)
    dtest = xgb.DMatrix(test_data_all_X, label=test_data_all_y)

    param = {
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.0, 1.0),
        "eta": trial.suggest_float("eta", 1e-8, 1.0),
        "gamma": trial.suggest_float("gamma", 1e-8, 1.0),
        "max_depth": trial.suggest_int("max_depth", 1, 9),
        "max_leaves": trial.suggest_int("max_leaves", 0, 50),
        "objective": "reg:squarederror",
        "reg_alpha": trial.suggest_loguniform("alpha", 1e-8, 2.0),
        "reg_lambda": trial.suggest_loguniform("lambda", 1e-8, 2.0),
        "eval_metric": "mae",
        "random_state" : 2
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-mae")
    best_model = xgb.train(param, dtrain, evals=[(dtest, "validation")], callbacks=[pruning_callback])
    preds = best_model.predict(dtest)
    pred_labels = np.rint(preds)
    accuracy = mean_absolute_error(test_data_all_y, pred_labels)
    # accuracy = { 'r2_score' : round(r2_score(test_data_all_y, pred_labels), 4), 'mse' :  round(mean_squared_error(test_data_all_y, pred_labels ),4), 'median abs err' :  round(median_absolute_error(test_data_all_y, pred_labels),4), 'mean abs err' :  round(mean_absolute_error(test_data_all_y, pred_labels),4)}
    return accuracy



In [39]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2022-04-08 10:11:08,572] A new study created in memory with name: no-name-315528cd-905d-445d-a67e-446c70270384


[0]	validation-mae:182.12363
[1]	validation-mae:172.77174
[2]	validation-mae:165.73238
[3]	validation-mae:156.70548
[4]	validation-mae:154.19252
[5]	validation-mae:153.07251
[6]	validation-mae:152.89870
[7]	validation-mae:152.94453
[8]	validation-mae:151.28181
[9]	validation-mae:151.28134


[I 2022-04-08 10:11:08,801] Trial 0 finished with value: 151.2803071859572 and parameters: {'booster': 'dart', 'colsample_bytree': 0.06926902275121016, 'eta': 0.883165156722046, 'gamma': 0.724122320905529, 'max_depth': 6, 'max_leaves': 46, 'alpha': 7.336828175554749e-07, 'lambda': 0.001455838076469405}. Best is trial 0 with value: 151.2803071859572.


[0]	validation-mae:75.95242
[1]	validation-mae:65.41508
[2]	validation-mae:60.65850
[3]	validation-mae:60.71726
[4]	validation-mae:60.25446
[5]	validation-mae:60.43544
[6]	validation-mae:61.96095
[7]	validation-mae:62.50245
[8]	validation-mae:62.63272
[9]	validation-mae:65.38998


[I 2022-04-08 10:11:09,055] Trial 1 finished with value: 65.38507953922107 and parameters: {'booster': 'gbtree', 'colsample_bytree': 0.9089659669004314, 'eta': 0.9749967096614847, 'gamma': 0.9193238816097603, 'max_depth': 7, 'max_leaves': 35, 'alpha': 1.0064876485480154e-06, 'lambda': 0.0014320554373507778}. Best is trial 0 with value: 151.2803071859572.


...

[0]	validation-mae:254.96417
[1]	validation-mae:254.88238
[2]	validation-mae:254.77470
[3]	validation-mae:254.68083
[4]	validation-mae:254.57413
[5]	validation-mae:254.47444
[6]	validation-mae:254.38649
[7]	validation-mae:254.29880
[8]	validation-mae:254.21503
[9]	validation-mae:254.13110


[I 2022-04-08 10:11:26,317] Trial 98 finished with value: 254.19418540866704 and parameters: {'booster': 'dart', 'colsample_bytree': 0.39051635215996094, 'eta': 0.0005299469431834816, 'gamma': 0.8933135693432322, 'max_depth': 3, 'max_leaves': 8, 'alpha': 1.4480209138302373e-05, 'lambda': 0.003044197173668727}. Best is trial 34 with value: 254.54854635216677.


[0]	validation-mae:240.61038


[I 2022-04-08 10:11:26,377] Trial 99 pruned. Trial was pruned at iteration 0.


FrozenTrial(number=34, values=[254.54854635216677], datetime_start=datetime.datetime(2022, 4, 8, 10, 11, 14, 890147), datetime_complete=datetime.datetime(2022, 4, 8, 10, 11, 15, 104541), params={'booster': 'dart', 'colsample_bytree': 0.4158466200732063, 'eta': 0.00025675381378648045, 'gamma': 0.9941895100340654, 'max_depth': 3, 'max_leaves': 12, 'alpha': 9.463978241232774e-07, 'lambda': 7.470523006997358e-07}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'colsample_bytree': UniformDistribution(high=1.0, low=0.0), 'eta': UniformDistribution(high=1.0, low=1e-08), 'gamma': UniformDistribution(high=1.0, low=1e-08), 'max_depth': IntUniformDistribution(high=9, low=1, step=1), 'max_leaves': IntUniformDistribution(high=50, low=0, step=1), 'alpha': LogUniformDistribution(high=2.0, low=1e-08), 'lambda': LogUniformDistribution(high=2.0, low=1e-08)}, user_attrs={}, system_attrs={}, intermediate_values={0: 255.007782, 1: 254.968048, 2: 254.915863, 3: 25

In [40]:
trial = study.best_trial
print(f'mae : {trial.value}')
print('best params :', trial.params)

mae : 254.54854635216677
best params : {'booster': 'dart', 'colsample_bytree': 0.4158466200732063, 'eta': 0.00025675381378648045, 'gamma': 0.9941895100340654, 'max_depth': 3, 'max_leaves': 12, 'alpha': 9.463978241232774e-07, 'lambda': 7.470523006997358e-07}


In [274]:
# All models 
xgb = xgboost.XGBRegressor(booster='gbtree', colsample_bytree = 0.493659, eta = 0.28, gamma=0,max_depth = 7, max_leaves=0, n_estimators =100, objective='reg:squarederror', random_state=32, 
                            reg_alpha=1.6666666666666667, reg_lambda =1.9791666666666667, subsample_freq=0.75, tree_method='auto', verbose=-1)

model_lgbm =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.7395, learning_rate=0.12, max_bin=1023, max_depth=8, reg_lambda=1,reg_alpha=0, subsample_freq=7, verbose=-1)

model_lgbm_2 =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.7385, learning_rate=0.07, max_bin=725, max_depth=8, reg_lambda=0.68596,reg_alpha=0, subsample_freq=56, verbose=-1)

model_lgbm_3 = lightgbm.LGBMRegressor(random_state=2, min_data_in_leaf=100, max_bin=1023, reg_alpha=0, subsample_freq=56, verbose=-1)

model_lgbm_4 =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.469, learning_rate=0.1239599, max_bin=1023, max_depth=8, reg_lambda=1,reg_alpha=0, subsample_freq=7,  verbose=-1)

VC = VotingRegressor([('xgboos' , xgb),('lgbm' , model_lgbm), ('lgbm2' , model_lgbm_2),('lgbm3' , model_lgbm_3), ('lgbm4' , model_lgbm_4)],)

stack_model = StackingRegressor([('xgboos' , xgb),('lgbm' , model_lgbm), ('lgbm2' , model_lgbm_2),('lgbm3' , model_lgbm_3),('lgbm4' , model_lgbm_4)])

stack_model_2 = StackingRegressor( [('1' , model_lgbm), ('3', xgb), ('4', VC)])

liste_model = [xgb, model_lgbm, model_lgbm_2, model_lgbm_3, model_lgbm_4, VC, stack_model, stack_model_2]

In [275]:
# Meilleur model : 
# Preprocessing et model xgboost :0.9056841627368329(8s)

logfunc = FunctionTransformer(func=np.log1p, inverse_func=np.exp, check_inverse=False)

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline(SimpleImputer(), MaxAbsScaler())
encode_pipeline = make_pipeline(StandardScaler(with_mean=True, with_std=False), OneHotEncoder(handle_unknown = 'ignore'))

preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))


model_selected = make_pipeline(preprocessor,  xgb)
model_selected.fit(train_data_all_X , train_data_all_y)
print('train data', model_selected.score(train_data_all_X, train_data_all_y))
print('test data', model_selected.score(test_data_all_X, test_data_all_y))

[13:32:53] WARNING: ../src/learner.cc:576: 
Parameters: { "subsample_freq", "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


train data 0.986720359083927
test data 0.8967965326020695


In [276]:
print(f'model stacking') 


print('r2 score train data', model_selected.score(train_data_all_X, train_data_all_y))
print('r2 score test data', model_selected.score(test_data_all_X, test_data_all_y))
y_pred = model_selected.predict(test_data_all_X)
print('mean squared error', mean_squared_error(test_data_all_y, y_pred ))
print('median abs error', median_absolute_error(test_data_all_y, y_pred))
print('mean abs error' , mean_absolute_error(test_data_all_y, y_pred))

model stacking
r2 score train data 0.986720359083927
r2 score test data 0.8967965326020695
mean squared error 4793.1408218943025
median abs error 32.69251251220703
mean abs error 48.643447046910445


# A ne pas touché ! Models saved avec les meilleurs score sans logarithme  

In [63]:
# Meilleur model : train data 0.9534144126106945
#                  test data 0.9222952589076324 ou 92.24 avec StackingRegressor( [('1' , model_lgbm), ('3', xgb), ('4', VC)])
# Preprocessing et model xgboost :0.9056841627368329(8s)

# All models 

xgb = xgboost.XGBRegressor(booster='gblinear', colsample_bytree = 0.493659, eta = 0.28, gamma=0,max_depth = 7, max_leaves=0, n_estimators =100, objective='reg:squarederror', random_state=2, 
                            reg_alpha=0.22936408146810935, reg_lambda = 0.03113496231279571, subsample_freq=0.75, tree_method='auto')


xgb2 = xgboost.XGBRegressor(booster='gbtree', colsample_bytree = 0.493659, eta = 0.28, gamma=0,max_depth = 7, max_leaves=0, n_estimators =100, objective='reg:squarederror', random_state=32, 
                            reg_alpha=1.6666666666666667, reg_lambda =1.9791666666666667, subsample_freq=0.75, tree_method='auto')

model_lgbm =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.7395, learning_rate=0.12, max_bin=1023, max_depth=8, reg_lambda=1,reg_alpha=0, subsample_freq=7)

model_lgbm_2 =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.7385, learning_rate=0.07, max_bin=725, max_depth=8, reg_lambda=0.68596,reg_alpha=0, subsample_freq=56)

model_lgbm_3 = lightgbm.LGBMRegressor(random_state=2, min_data_in_leaf=100, max_bin=1023, reg_alpha=0, subsample_freq=56)

model_lgbm_4 =  lightgbm.LGBMRegressor(random_state=2, colsample_bytree=0.469, learning_rate=0.1239599, max_bin=1023, max_depth=8, reg_lambda=1,reg_alpha=0, subsample_freq=7)

VC = VotingRegressor([('xgboos' , xgb),('lgbm' , model_lgbm), ('lgbm2' , model_lgbm_2),('lgbm3' , model_lgbm_3), ('lgbm4' , model_lgbm_4)])

stack_model = StackingRegressor([('xgboos' , xgb),('lgbm' , model_lgbm), ('lgbm2' , model_lgbm_2),('lgbm3' , model_lgbm_3),('lgbm4' , model_lgbm_4)])

stack_model_2 = StackingRegressor( [('1' , model_lgbm), ('3', xgb), ('4', model_lgbm_3)])

numerical_features = ['temp','atemp','humidity','windspeed','day','hour']
encode_column = ['season','workingday','weather','holiday','hour','year']

numerical_pipeline = make_pipeline(SimpleImputer(), MaxAbsScaler())
encode_pipeline = make_pipeline(SimpleImputer(), StandardScaler(with_mean=False, with_std=True), OneHotEncoder(handle_unknown = 'ignore'))


preprocessor = make_column_transformer((numerical_pipeline, numerical_features),(encode_pipeline , encode_column))


model_selected = make_pipeline(preprocessor,  stack_model_2) # CHoisi le modele ici 

model_selected.fit(train_data_all_X , train_data_all_y)

[10:16:45] WARNING: ../src/learner.cc:576: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "max_leaves", "subsample_freq", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:16:46] WARNING: ../src/learner.cc:576: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "max_leaves", "subsample_freq", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:16:47] WARNING: ../src/learner.cc:576: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "max_leaves", "subsample_fre

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('maxabsscaler',
                                                                   MaxAbsScaler())]),
                                                  ['temp', 'atemp', 'humidity',
                                                   'windspeed', 'day',
                                                   'hour']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                              

In [64]:
print(f'model selectionnée') 


print('r2 score train data', model_selected.score(train_data_all_X, train_data_all_y))
print('r2 score test data', model_selected.score(test_data_all_X, test_data_all_y))
y_pred = model_selected.predict(test_data_all_X)
print('mean squared error', mean_squared_error(test_data_all_y, y_pred ))
print('median abs error', median_absolute_error(test_data_all_y, y_pred))
print('mean abs error' , mean_absolute_error(test_data_all_y, y_pred))

model selectionnée
r2 score train data 0.9534109580772161
r2 score test data 0.9223042142455796
mean squared error 3608.4721936012556
median abs error 26.121466817633035
mean abs error 40.76929353338032


In [65]:
# import pickle
# filename = 'stacking.pkl'
# pickle.dump(model_selected, open(filename, 'wb'))